
# BrainCell 快速建模教程


## 一、基础元件建模
基础元件是构建神经元模型的核心单元，包含 **离子（Ion）** 和 **离子通道（IonChannel）**，直接决定神经元电生理特性（如动作电位产生、膜电流变化）。


### 1.1 离子建模
**作用**：
离子（如 Na⁺、K⁺、Ca²⁺）是神经元电活动的核心载体，其跨膜流动产生电流，驱动膜电位变化。
**核心功能**：
- 定义离子基本属性：名称、大小、通道、ion concertration、reversal potential。
- 支持与离子通道关联，建立通道与离子的选择性通透关系（如 Na⁺ 通道仅允许 Na⁺ 通过）。


### 1.2 离子通道建模
**作用**：
模拟细胞膜上选择性通透离子的蛋白结构，控制离子流动。
**核心功能**：
- 定义通道特性：关联离子类型、最大电导、门控动力学。
- 支持 Hodgkin-Huxley 型动力学。


## 二、房室建模
房室是模拟神经元空间结构的基本单元，可独立或组合使用，实现从单区域到全神经元的动态模拟。


### 2.1 单房室
**作用**：
简化模拟神经元局部区域，快速验证离子通道组合的电生理特性。
**核心功能**：
- 定义隔室物理参数：名称、大小、膜电容、阈值电压。
- 支持添加离子通道，构建“膜-通道”耦合系统，计算跨膜电流和膜电位变化。


### 2.2 多房室
**作用**：
模拟神经元复杂空间结构，实现了具有多个连接隔室的详细神经元模型，研究信号在神经元内的空间传播。
**核心功能**：
- 构建树状/链状房室网络：通过 `parent` 参数指定房室连接关系。
- 支持参数设置：不同隔室可定义独立的几何尺寸、电学特征，模拟真实神经元的形态异质性。


### 2.3 HH 模型建模
**作用**：
基于经典 Hodgkin-Huxley 理论的单房室模型，建模动作电位产生的完整过程。
**核心优势**：
- 开箱即用：
- 可扩展性：支持修改相关参数，用于验证不同离子机制对神经元兴奋性的影响。


## 三、微分方程建模
通过 **DiffEqState** 和 **DiffEqModule** 实现神经元动态的数学描述，将生理过程转化为状态变量演化的微分方程系统。


### 3.1 DiffEqState：状态变量管理
**作用**：
统一管理系统状态变量（如膜电位 `v`、通道门控变量 `m/h/n`），提供标准化的初始化、存储和访问接口。
**核心功能**：
- 定义多维度状态变量：支持常微分方程和随机微分方程，适配单神经元或网络模拟。
- 兼容 JAX 数组：无缝对接 JAX 自动微分和向量化计算，提升大规模模拟效率。


### 3.2 DiffEqModule：微分方程定义
**作用**：
定义状态变量的演化规则（即微分方程 `dx/dt = f(x, t)`），描述膜电位、门控变量随时间的变化规律。
**核心功能**：
- 实现梯度方法：输入当前状态和时间，返回各变量的导数（如 `dv/dt` 表示膜电位变化率）。
- 支持集成后处理：


## 快速上手建议
1. **从单房室开始**：使用 `HHNeuron` 类快速复现动作电位，熟悉基础工作流。
2. **模块化扩展**：逐步添加自定义离子通道或多房室结构，利用 BrainCell 的组件化设计降低复杂度。
3. **结合 JAX 加速**：通过 `jit` 优化模拟性能，尤其适用于大规模网络或参数扫描场景。

通过分层建模框架，``BrainCell`` 支持从分子机制（离子通道）到系统动态（神经元网络）的全尺度模拟，兼顾生物物理精确性与计算效率。

## 四、实例：兴奋性-抑制性神经元网络（E-I网络）的模拟与分析

以下将通过一个完整实例，展示如何基于前述基础元件和房室模型构建复杂网络，实现从单细胞到群体动态的层级建模。

### 准备工作
导入所需库（确保与前文环境一致）：

```python
import brainunit as u  # 神经科学单位系统
import matplotlib.pyplot as plt  # 可视化工具
import brainstate  # 网络动态管理
import braincell  # 核心建模库
```


### 1. 核心参数定义
定义神经元物理属性与网络规模参数：

```python
# 1.1 神经元电生理参数
V_th = -20. * u.mV  # 动作电位发放阈值（对应 SingleCompartment 的 V_th 参数）

# 膜面积（转换为 cm² 以匹配电导单位 mS/cm²）
area = 20000 * u.um **2  # 初始单位：平方微米
area = area.in_unit(u.cm** 2)  # 转换为平方厘米（1 cm² = 1e8 μm²）

# 膜电容（基于比电容 1 uF/cm²，总电容 = 比电容 × 面积）
Cm = (1 * u.uF * u.cm ** -2) * area  # 单位：pF
```


### 2. 构建 HH 单神经元模型
基于 `SingleCompartment` 类，整合钠通道、钾通道和漏电流：

```python
class HH(braincell.SingleCompartment):
    def __init__(self, in_size):
        # 初始化单房室：支持批量建模（in_size 为神经元数量）
        # 积分方法采用 ind_exp_euler（独立指数欧拉法，适合大规模网络）
        super().__init__(in_size, C=Cm, solver='ind_exp_euler', V_th=V_th)

        # 2.1 钠离子通道（INa）：基于 Traub & Miles 1991 模型
        self.na = braincell.ion.SodiumFixed(in_size, E=50. * u.mV)  # 钠反转电位 50 mV
        self.na.add_elem(
            # 最大电导 = 比电导（100 mS/cm²）× 膜面积
            INa=braincell.channel.INa_TM1991(
                in_size,
                g_max=(100. * u.mS * u.cm **-2) * area,
                V_sh=-63. * u.mV  # 电压偏移（修正通道激活曲线）
            )
        )

        # 2.2 钾离子通道（IK）：基于 Traub & Miles 1991 模型
        self.k = braincell.ion.PotassiumFixed(in_size, E=-90. * u.mV)  # 钾反转电位 -90 mV
        self.k.add_elem(
            IK=braincell.channel.IK_TM1991(
                in_size,
                g_max=(30. * u.mS * u.cm** -2) * area,  # 比电导 30 mS/cm²
                V_sh=-63. * u.mV
            )
        )

        # 2.3 漏电流（IL）：维持静息电位的非选择性电流
        self.IL = braincell.channel.IL(
            in_size,
            E=-60. * u.mV,  # 漏电流反转电位
            g_max=(5. * u.nS * u.cm **-2) * area  # 比电导 5 nS/cm²
        )
```


### 3. 构建 E-I 网络
组合 HH 神经元，定义兴奋性（E）和抑制性（I）突触连接：

```python
class EINet(brainstate.nn.DynamicsGroup):
    def __init__(self):
        super().__init__()
        # 3.1 网络规模：3200 兴奋性（E）+ 800 抑制性（I）神经元（比例 4:1，符合皮层特性）
        self.n_exc = 3200
        self.n_inh = 800
        self.total = self.n_exc + self.n_inh  # 总神经元数：4000

        # 3.2 初始化神经元群体（基于前文定义的 HH 模型）
        self.neurons = HH(self.total)

        # 3.3 兴奋性突触投射（E→所有神经元）
        self.exc_proj = brainstate.nn.AlignPostProj(
            # 连接规则：E 神经元以 2% 概率连接所有神经元
            comm=brainstate.nn.EventFixedProb(
                pre_size=self.n_exc,
                post_size=self.total,
                conn_num=0.02,  # 连接概率
                conn_weight=6. * u.nS  # 突触电导（兴奋性较弱）
            ),
            # 突触动力学：指数衰减（tau=5 ms，典型兴奋性突触）
            syn=brainstate.nn.Expon(self.total, tau=5. * u.ms),
            # 突触后效应：兴奋性（反转电位 0 mV）
            out=brainstate.nn.COBA(E=0. * u.mV),
            post=self.neurons
        )

        # 3.4 抑制性突触投射（I→所有神经元）
        self.inh_proj = brainstate.nn.AlignPostProj(
            # 连接规则：I 神经元以 2% 概率连接所有神经元
            comm=brainstate.nn.EventFixedProb(
                pre_size=self.n_inh,
                post_size=self.total,
                conn_num=0.02,
                conn_weight=67. * u.nS  # 抑制性突触更强，平衡兴奋性
            ),
            # 突触动力学：指数衰减（tau=10 ms，典型抑制性突触）
            syn=brainstate.nn.Expon(self.total, tau=10. * u.ms),
            # 突触后效应：抑制性（反转电位 -80 mV）
            out=brainstate.nn.COBA(E=-80. * u.mV),
            post=self.neurons
        )

    def update(self, t):
        # 3.5 网络动态更新规则
        with brainstate.environ.context(t=t):
            # 获取当前时刻的 spike 信号（布尔数组：True 表示发放）
            spikes = self.neurons.spike.value

            # 兴奋性神经元的 spike 驱动兴奋性突触
            self.exc_proj(spikes[:self.n_exc])  # 前 n_exc 个为 E 神经元

            # 抑制性神经元的 spike 驱动抑制性突触
            self.inh_proj(spikes[self.n_exc:])  # 剩余为 I 神经元

            # 更新神经元状态（无外部输入，仅靠突触驱动）
            new_spikes = self.neurons(0. * u.nA)
            return new_spikes
```


### 4. 运行网络模拟
配置仿真参数并执行模拟：

```python
# 4.1 初始化网络
net = EINet()
brainstate.nn.init_all_states(net)  # 初始化所有神经元和突触状态

# 4.2 设置仿真环境（时间步长 0.1 ms，高精度捕获 spike）
with brainstate.environ.context(dt=0.1 * u.ms):
    # 生成时间序列：0 ~ 100 ms，步长 dt
    times = u.math.arange(0. * u.ms, 100. * u.ms, brainstate.environ.get_dt())

    # 运行模拟（循环调用 update 函数，显示进度条）
    spikes = brainstate.compile.for_loop(
        net.update,
        times,
        pbar=brainstate.compile.ProgressBar(10)  # 进度条分 10 段
    )
```


### 5. 可视化网络动态
绘制spike 发放时间-神经元索引图：

```python
# 提取 spike 发生的时间和神经元索引
t_indices, n_indices = u.math.where(spikes)

# 绘制 raster plot
plt.figure(figsize=(10, 6))
plt.scatter(
    times[t_indices].to_decimal(u.ms),  # 横轴：时间（ms）
    n_indices,  # 纵轴：神经元索引
    s=1, color='k'  # 点大小为 1，避免重叠
)
plt.xlabel('时间（ms）')
plt.ylabel('神经元索引')
plt.title('E-I 网络的 Spike 发放模式')
plt.show()
```

通过本实例，你可以直观地理解不同层级在建模中的实际应用，
